======================== Import Packages ==========================

In [1]:
import sys, os, pdb, glob
import math
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from astropy.table import Table, join
import linmix
from astroquery.vizier import Vizier
import warnings
from astropy.logger import AstropyWarning
warnings.filterwarnings('ignore', category=AstropyWarning)

===================== Define Functions ===================

In [2]:
def get_data(catalog, join_key='Name', join_type='inner'):

    """
    PURPOSE:    Get data from literature with Vizier

    INPUT:      catalog = ctalog name on Vizier (str)
                join_key = column header to join tables, if multiple (str; optional)
                join_type = way to join tables, if multiple (str; optional)

    OUTPUT:     t = data table (AstroPy Table)

    """

    ### GET FULL CATALOG (ALL COLUMNS, ALL ROWS)
    viz = Vizier(catalog=catalog, columns=['**'])
    viz.ROW_LIMIT = -1
    tv = viz.get_catalogs(catalog)

    ### IF MULTIPLE TABLES, JOIN THEN
    for i, val in enumerate(tv.keys()):
        if i == 0:
            t = tv[val]
        else:
            tt = tv[val]
            if join_key in tt.columns:
                t = join(t, tt, join_type=join_type, keys=join_key)

    return t


In [3]:
def do_mc(x_data, x_err, y_data, y_err, ind_dd):

    ### MAKE DETECTION ARRAY (1=DET, 0=NON-DET)
    ddet = np.repeat(1, len(y_data))
    ddet[~ind_dd] = 0

    ### CALCULATE LINMIX FIT
    lmcens  = linmix.LinMix(x_data, y_data, x_err, y_err, delta=ddet, K=3)
    lmcens.run_mcmc(silent=True)
    A, Ae = np.mean(lmcens.chain['alpha']), np.std(lmcens.chain['alpha'])
    B, Be = np.mean(lmcens.chain['beta']), np.std(lmcens.chain['beta'])
    D, De = np.mean(np.sqrt(lmcens.chain['sigsqr'])), np.std(np.sqrt(lmcens.chain['sigsqr']))

    # print('\n =====================================')
    # print("\n LinMix_Err terms:")
    # print("    A  = {0:.2f}".format(A) + "+/- {0:.2f}".format(Ae))
    # print("    B  = {0:.2f}".format(B) + "+/- {0:.2f}".format(Be))
    # print("    D  = {0:.2f}".format(D) + "+/- {0:.2f}\n".format(De))

    ### SAVE TO FILE
    pname = '../output/linmix_mc'
    rn = len(glob.glob(pname + '/*rn.out'))
    fname = pname + "/" + "{0:02d}".format(rn) + '_rn.out'
    np.savetxt(fname, np.c_[lmcens.chain['alpha'], lmcens.chain['beta'], lmcens.chain['sigsqr']], fmt='%1.2e')

    pars = np.array([A, Ae, B, Be, D, De])

    return pars, x_data, x_err

In [4]:
def assign_mstar(ms, e_ms):

    # INDEX STARS WITHOUT MASS MEASUREMENTS
    f_ms = np.copy(ms.mask)
    ind_nomass = np.where(ms.mask == True)

    ### DISTRIBUTION OF LUPUS STELLAR MASSES
    ### FROM Mortier+2011 (2011MNRAS.418.1194M) FIGURE 9
    hist_values = np.array([ 5., 5., 12., 12., 12., 9., 9., 9., 1., 1., 1., 1., 1., 1.])
    log_mstar_bins = np.array([-1.05, -0.95, -0.85, -0.75, -0.65, -0.55, -0.45, 
                               -0.35, -0.25, -0.15, -0.05,  0.05,  0.15,  0.25])
    
    ### RANDOMLY ASSIGN VALUE FROM RANGE SEEN IN LUPUS (MORTIER+2011)
    ### USE MEDIAN FRACTIONAL ERROR OF MSTAR FOR LUPUS SOURCES WITH KNOWN MSTAR 
    mstar_probs = hist_values / np.sum(hist_values)
    randm = 10**(np.random.choice(log_mstar_bins, len(ind_nomass[0]), p=list(mstar_probs)))
    for i, val in enumerate(ind_nomass[0]):
        ms[val] = "{0:.2f}".format(randm[i])
        e_ms[val] = "{0:.2f}".format(randm[i] * .20) 

    return ms, e_ms, ~f_ms

============================= Code ==================================

In [5]:
### MAKE OUTPUT DIRECTORY IF DOESN'T EXIST
mc_dir = '../output/linmix_mc'
if not os.path.exists(mc_dir):
    os.makedirs(mc_dir)

In [ ]:
### DO MC RUNS
if __name__ == '__main__':

    A, Ae, B, Be, D, De = [], [], [], [], [], []
    for i, val in enumerate(np.arange(0,100)):

        #### LOAD IN LUPUS DATA
        T = get_data("J/ApJ/828/46")

        #### GET STELLAR MASSES FOR THOSE WITH UNKNOWN VALUES
        mstar, e_mstar, ind_mstar = assign_mstar(T['Mass'], T['e_Mass'])
        T['Mass'], T['e_Mass'] = mstar, e_mstar

        ### INDEX (NON-)DETECTIONS
        ind_dd = T['F890'] / T['e_F890'] >= 3.0
        ind_nd = T['F890'] / T['e_F890'] < 3.0

        ### DEFINE PLOT VARIABLES
        x_data_lin = np.copy(T['Mass'])
        y_data_lin = np.copy(T['MDust'])
        y_data_lin[ind_nd] = 3.0 * np.copy(T['e_MDust'][ind_nd])
        y_err_lin = np.sqrt((T['e_MDust'])**2 + (0.1 * T['e_MDust'])**2)
        x_err_lin = np.copy(T['e_Mass'])

        ### CONVERT TO LOG SCALE
        x_data = np.log10(x_data_lin)
        y_data = np.log10(y_data_lin)
        y_err  = np.array(0.434 * (y_err_lin / y_data_lin))
        x_err  = np.array(0.434 * (x_err_lin / x_data_lin))

        ### GET MCMC FIT WITH LINMIX
        pars, x_data, x_err = do_mc(x_data, x_err, y_data, y_err, ind_dd)
        a, ae, b, be, d, de = pars[0], pars[1], pars[2], pars[3], pars[4], pars[5]
        A.append(a)
        Ae.append(ae)
        B.append(b)
        Be.append(be)
        D.append(d)
        De.append(de)